Test for create_daily_share_quantity()
######################################

Notes:
- Function requires create_share_quantity_df(). Rename -> create_empty_share_quantity_df()
- Create_share_quantity_df() produces a df with 'Date' label in index column. -> Ok 
- create_daily_share_quantity() removes 'Date' label in index. -> Add
- create_daily_share_quantity() dates are in ascending order (last day first)  -> Make descending
- create_daily_share_quantity() shares are object dtypes  -> Make Integers
- Review if its worth changing the date column so its not the index


In [8]:
import unittest
import pandas as pd
from pandas import Timestamp
from scripts import daily_shares as ds


In [9]:
# Setup
date_range1 = pd.date_range(start="2022-12-01", end="2022-12-06")

dates1 = [
    Timestamp('2022-12-01 00:00:00'),
    Timestamp('2022-12-03 00:00:00'),
    Timestamp('2022-12-05 00:00:00'),
    ]

df1 = pd.DataFrame({
    'Ticker': ['VOO', 'VOO', 'VOO'],
    'Yfinance_Ticker': ['VOO.MX', 'VOO.MX', 'VOO.MX'],
    'Trade': ['Buy', 'Buy', 'Sell'],
    'Shares': [10, 5, -3],
    }, index=dates1)

df1 = df1.rename_axis('Date', axis=1)

yftickers = list(df1['Yfinance_Ticker'].unique())
df2 = ds.create_share_quantity_df(yftickers)


In [10]:
df1

Date,Ticker,Yfinance_Ticker,Trade,Shares
2022-12-01,VOO,VOO.MX,Buy,10
2022-12-03,VOO,VOO.MX,Buy,5
2022-12-05,VOO,VOO.MX,Sell,-3


In [11]:
df2

Date,VOO.MX


In [14]:

# Call function
actual = ds.create_daily_share_quantity(
    date_range=date_range1,
    trade_history=df1,
    ticker_list=yftickers,
    df=df2)
actual
#print(actual.dtypes)

,VOO.MX
2022-12-06,12
2022-12-05,12
2022-12-04,15
2022-12-03,15
2022-12-02,10
2022-12-01,10


In [15]:

# Expected

dates_expected2 = pd.date_range('2022-12-01', periods=6)

dates_expected = [
    Timestamp('2022-12-01 00:00:00'),
    Timestamp('2022-12-03 00:00:00'),
    Timestamp('2022-12-05 00:00:00'),
    ]

expected = pd.DataFrame({
    'VOO.MX': [10, 10, 15, 15, 12, 12],
    }, index=dates_expected2)
expected.sort_index(ascending=False, inplace=True)

expected
print(expected.dtypes)

VOO.MX    int64
dtype: object


In [ ]:
a = pd.testing.assert_frame_equal(actual, expected, check_freq=False)